In [1]:
import datetime
import sys
import warnings

import jax
import jax.numpy as jnp
from get_model import get_model_and_data  # noqa: F401
from main import run_simple_lmc_numpyro  # noqa: F401
from numpyro.infer import MCMC, NUTS, Predictive  # noqa: F401
from progressive_full_run import run_progressive_logreg


%env JAX_PLATFORM_NAME=cuda
warnings.simplefilter("ignore", FutureWarning)

jnp.set_printoptions(precision=3, suppress=True, threshold=sys.maxsize)
jax.config.update("jax_enable_x64", True)
print(jax.devices("cuda"))

names = [
    "banana",
    "breast_cancer",
    "diabetis",
    "flare_solar",
    "german",
    "heart",
    "image",
    "ringnorm",
    "splice",
    "thyroid",
    "titanic",
    "twonorm",
    "waveform",
]

env: JAX_PLATFORM_NAME=cuda
[CudaDevice(id=0)]


In [2]:
import glob


timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"progressive_results/log_{timestamp}.txt"
with open(log_filename, "w") as f:
    f.write(f"Results for time {timestamp}\n\n")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
for name in names:
    dict_filename = glob.glob(f"progressive_results/result_dict_{name}_*.pkl")[0]
    print(f"==================== {name} ====================")
    run_progressive_logreg(
        name,
        log_filename,
        timestamp,
        nuts_dict_filename=dict_filename,
        quic_dict_filename=dict_filename,
    )
    print()

==================== banana ====================
Data shape: (5300, 2)
GT energy bias: 1.374e-05, test acc: 0.5468, test acc top 90%: 0.5547
QUICSORT acc: 0.5517, acc top 90%: 0.557, energy: 5.419e-05, w2: 3.961e-04


100.00%|██████████| [00:37<00:00,  2.68%/s]


Euler acc: 0.5111, acc top 90%: 0.5226, energy: 5.581e-01, w2: 1.345e+00
NUTS acc: 0.5509, acc top 90%: 0.5568, energy: 7.520e-06, w2: 3.826e-04

==================== breast_cancer ====================
Data shape: (263, 9)
GT energy bias: 1.405e-04, test acc: 0.6599, test acc top 90%: 0.6655
QUICSORT acc: 0.6651, acc top 90%: 0.6713, energy: 2.021e-04, w2: 1.483e-01


100.00%|██████████| [00:30<00:00,  3.31%/s]


Euler acc: 0.6643, acc top 90%: 0.671, energy: 1.308e-02, w2: 2.087e-01
NUTS acc: 0.6641, acc top 90%: 0.6726, energy: 2.228e-04, w2: 1.490e-01

==================== diabetis ====================
Data shape: (768, 8)
GT energy bias: 9.340e-05, test acc: 0.7777, test acc top 90%: 0.7802
QUICSORT acc: 0.7786, acc top 90%: 0.7812, energy: 1.394e-04, w2: 4.334e-02


...

Euler acc: 0.7298, acc top 90%: 0.7384, energy: 6.179e-01, w2: 1.774e+00
NUTS acc: 0.7788, acc top 90%: 0.7814, energy: 1.338e-04, w2: 4.328e-02

==================== flare_solar ====================
Data shape: (144, 9)
GT energy bias: 3.103e-03, test acc: 0.6165, test acc top 90%: 0.6244
QUICSORT acc: 0.6295, acc top 90%: 0.6385, energy: 6.967e-02, w2: 2.117e+00


100.00%|██████████| [00:29<00:00,  3.41%/s]


Euler acc: 0.6071, acc top 90%: 0.6137, energy: 3.786e+00, w2: 6.533e+01
NUTS acc: 0.6191, acc top 90%: 0.6269, energy: 1.461e-02, w2: 3.028e+00

==================== german ====================
Data shape: (1000, 20)
GT energy bias: 3.190e-04, test acc: 0.7838, test acc top 90%: 0.7875
QUICSORT acc: 0.7848, acc top 90%: 0.7884, energy: 3.173e-04, w2: 1.614e-01


37.45%|███▋      | [00:12<00:21,  2.93%/s]E0930 13:33:11.360351   20448 pjrt_stream_executor_client.cc:3067] Execution of replica 0 failed: INTERNAL: CustomCall failed: CpuCallback error: Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, 

XlaRuntimeError: INTERNAL: CustomCall failed: CpuCallback error: Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
  File "/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
  File "/home/andy/PycharmProjects/diffrax_ST